In [1]:
#!/usr/bin/env python
"""
Demo for Emoji dynamic tomography test problem
--------------------------------------------------------------------------
Created in 2023 for TRIPs-Py library
"""
__authors__ = "Mirjeta Pasha, Silvia Gazzola, Connor Sanderford, and Ugochukwu Obinna Ugwu"
__affiliations__ = 'Tufts University, University of Bath, Arizona State University, and Tufts University'
__copyright__ = "Copyright 2023, TRIPs-Py library"
__license__ = "GPL"
__version__ = "1.0"
__email__ = "mirjeta.pasha@tufts.edu; mirjeta.pasha1@gmail.com; sg968@bath.ac.uk; csanderf@asu.edu; connorsanderford@gmail.com; Ugochukwu.Ugwu@tufts.edu"

In [2]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from scipy.linalg import fractional_matrix_power
from numpy import array, diag, dot, maximum, empty, repeat, ones, sum
from numpy.linalg import inv
parameters = {'xtick.labelsize': 12, 'ytick.labelsize': 12,
          'axes.titlesize': 18, 'axes.labelsize': 18, 'figure.titlesize': 14, 'legend.fontsize': 13}
plt.rcParams.update(parameters)
import scipy as sp
import scipy.stats as sps
import scipy.io as spio
import astra
from venv import create
import pylops
from scipy.ndimage import convolve
from scipy import sparse
from scipy.ndimage import convolve
import scipy.special as spe
import requests
import h5py
# functions to generate emoji data are stored in io_l.py
from trips.utilities.io import *
from trips.utilities.operators import *
from trips.solvers.AnisoTV import *
from trips.utilities.helpers import *
from trips.solvers.GKS import *
from trips.solvers.MMGKS import *
from trips.solvers.MMGKS_new import *

# Notebook 1: Emoji test problem

<p>In this example we illustrate the following:
<p>&nbsp; Step 1: Download/read the emoji data and prepare them for processing.
<p>&nbsp; Step 2: Solve the static inverse problem.
<p>&nbsp; Step 3: Solve the dynamic inverse problem.

Emoji data are generated at the University of Helsinki: https://zenodo.org/record/1183532#.YyaW_C9h2fU


#### Step 1: Download/read the data and prepare them for processing
##### The main function to generate the emoji data is generate_emoji(dataset, noise_level) 
<p> <b>Input:</b> 
<p>&nbsp; <b>dataset:</b>
        <p>&nbsp; if dataset = 30, we generate data from DataDynamic_128x30.mat
        <p>&nbsp; if dataset = 60, we generate data from DataDynamic_128x60.mat
<p>&nbsp; <b>noise_level:</b>
        <p>&nbsp; Emoji data are real data and they have noise. Hence, we set noise_level = 0
            
<p> <b>Output:</b>  
&nbsp;<br> <b>A:</b> The full forward operator for all time instances<br>
&nbsp; <b>b:</b> All the data/measurements vectorized<br> 
&nbsp; <b>AA:</b> A list of 33 time-wise operators<br> 
&nbsp; <b>B:</b> A list of 33 data vectors for each time instance <br> 
&nbsp; <b>nx:</b> The dimension of the first spatial domain<br> 
&nbsp; <b>ny:</b> The dimension of the second spatial domain<br> 
&nbsp; <b>nt:</b> The dimension of the time instances domain<br> 
&nbsp; <b>delta:</b> The noise level in the data. When we set noise_level == 0, we get delta = 0<br> 
&nbsp; <b>sinoshape:</b> The dimension of the time-wise sinogram<br>     

In [3]:
(A, b, AA, B, nx, ny, nt, delta) = generate_emoji(noise_level = 0, dataset = 30)

data already downloaded.


#### Step 2: Solve the static inverse problems.
<p>&nbsp; <b>The function to perform reconstruction is reconstruct_anisoTV(dynamic)</b>
<p> <b>Input:</b> 
<p>&nbsp; <b>dynamic:</b>
        <p>&nbsp; if dynamic = True, it solves the dynamic inverse problem $A x = b$
        <p>&nbsp; if dynamic = False, it solves nt static inverse problems $AA(i) x(i) = B(i)$, for i in range(nt)
<p> <b>Output:</b>  
&nbsp;<br> <b>xx:</b> <br>If dynamic == True, xx is a 3 dimensional tensor with dimensions (nx, ny, nt) that contains nt reconstructed images<br>If dynamic == False, xx is a list with nt elements that contains nt reconstructed images

In [4]:
# Compute a solution by GKS
# Define a derivative operator as a regularization operator
# L = spatial_derivative_operator(nx, ny, nt)
A1 = AA[1]
b1 = B[1]
L = gen_first_derivative_operator_2D(nx, ny)
data_vec = b.reshape((-1,1))
# Use GKS to compute an approximate solution
(x_mmgks_1a, info_mmgks_1a) = MMGKS(A1, b1.reshape((-1,1)), L, pnorm=2, qnorm=1, projection_dim=1, n_iter = 50, regparam = 'gcv', x_true = None, delta = delta, epsilon = 0.001)
(x_mmgks_1b, info_mmgks_1b) = MMGKS_new(A1, b1.reshape((-1,1)), L, pnorm=2, qnorm=1, projection_dim=1, n_iter = 50, regparam = 'gcv', x_true = None, delta = delta, epsilon = 0.001)
print(info_mmgks_1a['regParam'])
print(info_mmgks_1b['regParam'])

running MMGKS...: 100%|██████████| 50/50 [00:03<00:00, 16.36it/s]


In [ ]:
xxa = list(range(nt))
regp_vara = list(range(nt))
L = spatial_derivative_operator(nx, ny, 1)
for i in range(nt):
    b_vec = B[i].reshape((-1,1))
    (x_static_mmgks, info_mmgks) = MMGKS(AA[i], b_vec, L, pnorm=2, qnorm=1, projection_dim=1, n_iter = 10, regparam= 'gcv' , x_true=None, epsilon = 0.001)
    xxa[i] = x_static_mmgks
    regp_vara[i] = info_mmgks['regParam']

xxb = list(range(nt))
regp_varb = list(range(nt))
L = spatial_derivative_operator(nx, ny, 1)
for i in range(nt):
    b_vec = B[i].reshape((-1,1))
    (x_static_mmgks, info_mmgks) = MMGKS_new(AA[i], b_vec, L, pnorm=2, qnorm=1, projection_dim=1, n_iter = 10, regparam= 'gcv' , x_true=None, epsilon = 0.001)
    xxb[i] = x_static_mmgks
    regp_varb[i] = info_mmgks['regParam']

In [7]:
print(regp_vara)
print(regp_varb)

[0.8243788017822309, 0.843693496731194, 0.7597867904634314, 0.8823602486947697, 0.0015440138904118023, 0.0013721125400948516, 0.0011423054771074663, 0.0013097572120293354, 0.001319287907457819, 0.0012463085210955304, 0.001569403795337984, 0.0016151576402765962, 0.0011353855459415602, 0.0012442927710000684, 0.0012775237609876427, 0.0012192085756170701, 0.0012870495394483228, 0.001568901968165609, 0.001176223149701261, 0.0013071871383326843, 0.0013199979872624194, 0.8690017600810329, 0.001643363749006486, 0.0013848406109833648, 0.8460660101623654, 0.0011768664464686198, 0.0013066476821117997, 0.0011815435473319469, 0.0015616935879171699, 0.0015305767747801485, 0.7323364195041993, 0.0011205553619839483, 0.0014346433825539625]


In [ ]:
plot_recstructions_series(xxa, (nx, ny, nt), dynamic = False, testproblem = 'Emoji', geome_x = 1,geome_x_small = 0,  save_imgs= False, save_path='./reconstruction/Emoji')

In [ ]:
plot_recstructions_series(xxb, (nx, ny, nt), dynamic = False, testproblem = 'Emoji', geome_x = 1,geome_x_small = 0,  save_imgs= False, save_path='./reconstruction/Emoji')

In [ ]:
plt.imshow(x_mmgks_1a.reshape((nx, ny)))

In [ ]:
plt.imshow(x_mmgks_1b.reshape((nx, ny)))

#### Step 3: Solve dynamic inverse problems.
<p>&nbsp; <b>The function to perform reconstruction is reconstruct_anisoTV(dynamic)</b>
<p> <b>Input:</b> 
<p>&nbsp; <b>dynamic:</b>
        <p>&nbsp; if dynamic = True, it solves the dynamic inverse problem $A x = b$
        <p>&nbsp; if dynamic = False, it solves nt static inverse problems $AA(i) x(i) = B(i)$, for i in range(nt)
<p> <b>Output:</b>  
&nbsp;<br> <b>xx:</b> <br>If dynamic == True, xx is a 3 dimensional tensor with dimensions (nx, ny, nt) that contains nt reconstructed images<br>If dynamic == False, xx is a list with nt elements that contains nt reconstructed images

In [ ]:
# Compute a solution by GKS
# Define a derivative operator as a regularization operator
# L = spatial_derivative_operator(nx, ny, nt)
L = gen_spacetime_derivative_operator(nx, ny, nt)
data_vec = b.reshape((-1,1))
# Use GKS to compute an approximate solution
(x_mmgks, info_mmgks) = MMGKS(A, data_vec, L, pnorm=2, qnorm=1, projection_dim=1, n_iter = 50, regparam = 0.01, x_true = None, delta = delta, epsilon = 0.001)

In [ ]:
(x_mmgks_var, info_mmgks_var) = MMGKS(A, data_vec, L, pnorm=2, qnorm=1, projection_dim=1, n_iter = 50, regparam = 'gcv', x_true = None, delta = delta, epsilon = 0.001)

In [ ]:
print(info_mmgks_var)

In [ ]:
x_mmgks[x_mmgks<0] = 0

In [ ]:
plot_recstructions_series(x_mmgks, (nx, ny, nt), dynamic = True, testproblem = 'Emoji', geome_x = 1,geome_x_small = 0,  save_imgs= False, save_path='./reconstruction/Emoji')

In [ ]:
x_mmgks_var[x_mmgks_var<0] = 0

In [ ]:
plot_recstructions_series(x_mmgks_var, (nx, ny, nt), dynamic = True, testproblem = 'Emoji', geome_x = 1,geome_x_small = 0,  save_imgs= False, save_path='./reconstruction/Emoji')